In [2]:
from bs4 import BeautifulSoup
import json
import os
import pandas as pd
import sqlite3

In [4]:
names_list = []
favorites_list = []

#for name in folder:

directory = os.fsencode('/Users/gretagoss/Downloads/DATA_304/test/data-304/data/assignment_3/downloaded_class_submissions')

data_dir = '/Users/gretagoss/Downloads/DATA_304/test/data-304/data/assignment_3/downloaded_class_submissions'
#    html_file = os.path.join(data_dir, 'hello_class_vu.html') #need to figure out how to iterate through files

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".html"):
        html_file = os.path.join(data_dir, filename)
        with open(html_file, 'r') as file:
            soup = BeautifulSoup(file, 'html.parser')
    
        # Extract name
        h1_tag = soup.find('h1')
        name_value = h1_tag.text.strip()
        names_list.append(name_value)
        
        table = soup.find('table')
        
        # Extract table headers
        headers = [th.text.strip() for th in table.find_all('th')]
        
        # Add the "name" column to the headers
        headers.append('Name')
        
        # Extract table rows and add the "name" column value
        for row in table.find_all('tr')[1:]:  # Skipping the header row
            cols = [td.text.strip() for td in row.find_all('td')]
            cols.append(name_value)  # Add the name column value
            favorites_list.append(cols)

df_favorites = pd.DataFrame(favorites_list, columns=headers)
df_favorites['Favorite_id'] = df_favorites.index + 1

df_names = pd.DataFrame(names_list, columns=['Name'])
df_names['Name_id'] = df_names.index + 1

In [6]:
os.listdir(directory)

[b'A1.jpg',
 b'elias.png',
 b'.DS_Store',
 b'vu_photo.JPG',
 b'hello_class_sagegray.html',
 b'hello_class_bradyseng.html',
 b'Untitled.ipynb',
 b'hello_class_kebba.html',
 b'IMG_6969.png',
 b'hello_class_mateo.html',
 b'hello_class_jay.html',
 b'HTMLPicture_Purohit.jpg',
 b'hello_class_gretagoss.html',
 b'just_me.png',
 b'data_photo.jpeg',
 b'hello_class_Elias_Brown.html',
 b'IMG_1359.jpeg',
 b'hello_class_Weis.html',
 b'.ipynb_checkpoints',
 b'sushi.jpg',
 b'pic.jpg',
 b'hello_class_abhipurohit.html',
 b'hello_class_vu.html',
 b'hello_class_nathanebbs.html']

In [8]:
directory

b'/Users/gretagoss/Downloads/DATA_304/test/data-304/data/assignment_3/downloaded_class_submissions'

In [10]:
df_favorites = df_favorites.merge(df_names, how='left')
df_favorites = df_favorites.drop(columns=['Name'])
df_favorites = df_favorites[['Favorite_id','Name_id', 'Category','Favorite']]
df_favorites

,Favorite_id,Name_id,Category,Favorite
0,1,1,Ice Cream,Butterscotch
1,2,1,Sport,Rock climbing
2,3,1,Pass time,Board Games!
3,4,2,Food,Sushi
4,5,2,Sport,Baseball
5,6,2,Hobby,Golf
6,7,3,Ice Cream,Oreo
7,8,3,Sport,Soccer
8,9,3,Pass time,Reading
9,10,4,Show,Ozark


In [12]:
df_names = df_names[['Name_id','Name']]
df_names

,Name_id,Name
0,1,Sage Gray
1,2,Brady Sengkhounmany
2,3,Kebba Leigh
3,4,Mateo Marneau
4,5,Jay Naidu
5,6,Greta Goss
6,7,Elias Brown
7,8,Brandon Weis
8,9,Abhi Purohit
9,10,Vu Mai


In [41]:
df_favorites.to_csv('favorites', index=False, encoding='utf-8')
df_names.to_csv('names', index=False, encoding='utf-8')

In [16]:
conn = sqlite3.connect("parsed_bios.sqlite")

df_favorites.to_sql("favorites", conn, if_exists="replace", index=False)  # Replace "table1" with your table name
df_names.to_sql("names", conn, if_exists="replace", index=False)

query = "SELECT * FROM favorites LIMIT 5;"
df_check = pd.read_sql(query, conn)
print(df_check)

conn.close()

   Favorite_id  Name_id   Category       Favorite
0            1        1  Ice Cream   Butterscotch
1            2        1      Sport  Rock climbing
2            3        1  Pass time   Board Games!
3            4        2       Food          Sushi
4            5        2      Sport       Baseball
